# Video Processing for BT Lab
Converts a video into png files. 
<br>
<br>
Usage: run notebook 
<br>
<br>
Notes: looks for a couple of JSON files to find video data.
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [69]:
#import libraries
import os
import re
import cv2
import json
import pandas as pd
import numpy as np
from timeit import default_timer as timer

In [70]:
user_drive = input("Enter user drive: ")
video_path = f"{user_drive}:/Christian/DI_centre_structured"
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"

In [71]:
def set_folder(save_folder:str) -> None:
    if not os.path.isdir(save_folder):
        os.mkdir(save_folder)
    else:
        for files in os.listdir(save_folder):
            os.remove(os.path.join(save_folder, files))

In [72]:
def run_video_to_frame(video_path:str , save_folder:str, frame_frequency: dict, new_fps:int) -> None:
    video = cv2.VideoCapture(video_path)
    assert video.isOpened()
    vid_fps = int(video.get(cv2.CAP_PROP_FPS))
    number_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    vid_length_floored = number_of_frames // vid_fps

    pick_counter, save_counter, set_counter, true_frames = [0, 0, 0, 0]
    list_of_frames = [frame for frame, frequency in frame_frequency.items()]
    expected_frames = new_fps * vid_length_floored
    success = True
    frame_limit = 100

    #clean or make folder
    set_folder(save_folder)
    
    print(f"\nSaving frames to: {save_folder}\nPicking frames: {list_of_frames} per set\nExpected number of frames: {expected_frames} ({new_fps}FPS * {vid_length_floored}s).")
    
    start_time  = timer()
    while success and set_counter < vid_length_floored:
        success, frame = video.read()

        if not success:
            break

        if pick_counter in list_of_frames:
            frame_name = "frame_%d.png" % true_frames
            frame_path = os.path.join(save_folder, frame_name)
            num_of_save = frame_frequency.get(pick_counter)
            
            for _ in range(num_of_save):
                # print(f"Saving frame: {frame_name}")
                # cv2.imwrite(frame_path, frame) 
                save_counter += 1

        pick_counter += 1
        true_frames += 1

        if (pick_counter == vid_fps):
            set_counter += 1
            pick_counter = 0

        # for testing
        # if true_frames > frame_limit:
        #     print(f"\nLimit set to {frame_limit} frames for testing purposes.")
        #     break
    
    end_time  = timer()
    time_delta = end_time - start_time
    print(f"\nDone in {time_delta} seconds.")
    video.release()

    if save_counter != expected_frames:
        raise ValueError(f"Expected {expected_frames} frames, but got {save_counter} frames")

    return [set_counter, save_counter, true_frames]

In [73]:
def get_frames_path(local_path: str, level:str) -> str:
    fixed_path = local_path.replace("\\", "/")
    fixed_path_split = fixed_path.split("/")
    video_folder = "/".join(fixed_path_split[:-1])
    video_filename = fixed_path_split[-1].split(".")[0]
    folder_path = video_folder + f"/frames_{video_filename}_{level}"
    return folder_path

In [74]:
def up_sample(old_fps: int, new_fps: int) -> list[int]:
    frames_arr = np.arange(0, old_fps, dtype=int)
    frames_interp = np.linspace(0, old_fps - 1, new_fps)
    nearest_indices = np.round(frames_interp).astype(int)
    up_sampled_list = np.take(frames_arr, nearest_indices, mode='wrap')

    return up_sampled_list.tolist()

In [75]:
def convert_video_to_frame(all_patients:dict, level: str, new_fps:int) -> None:
    for json_index, patient_info in all_patients.items():
        try:
            video_path = patient_info["local path"]
            old_fps = int(patient_info["old fps"])

            frames_folder = get_frames_path(video_path, level)
            
            frames_to_pick = up_sample(old_fps, new_fps)
            frames_idx = pd.Index(frames_to_pick, name="frames")
            frame_frequency = frames_idx.value_counts()
            
            if len(frames_to_pick) != new_fps:
                raise ValueError("Number of frames to pick is not equal to new fps")

            # frame_frequency = get_frequency(frames_to_pick)
            set_counter, save_counter, true_frames = run_video_to_frame(video_path, frames_folder, frame_frequency, new_fps)
            print(f"\nSet counter: {set_counter}, save counter: {save_counter}, frame counter: {true_frames}\n\n"+ "-"*50)
        except Exception as e:
            print(f'''{type(e)}: {e} for video {patient_info["filename"]}''')

        # break

In [76]:
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

## Video to frames (stage 3.1)

Based on the desired fps, turn videos into frames

In [77]:
""" local vals"""

rgb_fps = {
    "lower_bound": 10,
    "upper_bound": 20
}

thermal_fps = {
    "lower_bound": 5,
    "upper_bound": 10
}

In [78]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "rgb_test.json")
metadata_thermal = load_json(json_dir, "thermal_test.json")

In [79]:
""" convert video to png (rgb) """

for level, new_fps in rgb_fps.items():
    print(f"\nAdjusting FPS to {new_fps}\n" + "="*50)
    convert_video_to_frame(metadata_rgb, level, new_fps)
    # break


Adjusting FPS to 10

Saving frames to: E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/frames_Arun2_lower_bound
Picking frames: [0, 3, 6, 9, 12, 16, 19, 22, 25, 28] per set
Expected number of frames: 740 (10FPS * 74s).



Done in 6.996264699999301 seconds.

Set counter: 74, save counter: 740, frame counter: 2146

--------------------------------------------------

Saving frames to: E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/frames_Arun2_lower_bound
Picking frames: [0, 2, 4, 6, 8, 9, 11, 13, 15, 17] per set
Expected number of frames: 70 (10FPS * 7s).

Done in 0.42374800000106916 seconds.

Set counter: 7, save counter: 70, frame counter: 126

--------------------------------------------------

Adjusting FPS to 20

Saving frames to: E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/frames_Arun2_upper_bound
Picking frames: [0, 1, 27, 25, 24, 22, 21, 19, 18, 16, 15, 13, 12, 10, 9, 7, 6, 4, 3, 28] per set
Expected number of frames: 1480 (20FPS * 74s).

Done in 6.797871900000246 seconds.

Set counter: 74, save counter: 1480, frame counter: 2146

--------------------------------------------------

Saving frames to: E